In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [4]:
FOLD = 1

In [5]:
EXP_PATH = Path.cwd()
ROOT_DIR = EXP_PATH.parents[2]
exp_name = EXP_PATH.name
RAW_DATA_DIR = ROOT_DIR / 'data' / 'original_data'
DATA_DIR = ROOT_DIR / 'data' / 'kaggle_dataset' / 'irohith_tfrecords'
SAVE_DIR = ROOT_DIR / 'outputs' / exp_name / f'fold{FOLD}'
SAVE_DIR.mkdir(parents=True, exist_ok=True)

In [6]:
valid_result_df = pd.read_csv(SAVE_DIR / 'valid_result.csv')

In [21]:
valid_result_df = valid_result_df.fillna('').astype(str)

In [22]:
from Levenshtein import distance as Lev_distance
def calculate_N_D(s1, s2):
    l = len(s1)
    lvd = Lev_distance(s1, s2)
    return lvd, l
    
global_N, global_D = 0, 0
for target_strings, predict_strings in valid_result_df[['target', 'pred_epoch5']].values:
    values = [calculate_N_D(target, predict) for target, predict in zip(target_strings, predict_strings)]
    global_D += np.sum([x[0] for x in values])
    global_N += np.sum([x[1] for x in values])

metric_value = np.clip((global_N - global_D) / global_N, a_min=0, a_max=1)

In [23]:
metric_value

0.3132487642017859